In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/colabCalisma/nucleiUnet/"

/content/drive/MyDrive/colabCalisma/nucleiUnet


In [ ]:
import time
import tensorflow as tf
import os
import random
import numpy as np

from tqdm import tqdm

from skimage.io import imread, imshow, imsave
from skimage.transform import resize
import matplotlib.pyplot as plt



IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

TEST_PATH = 'train_im_label/'


In [ ]:
tf.keras.__version__

'2.4.0'

In [ ]:
X_train = np.zeros((536, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((536, IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
X_test = np.zeros((134, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((134, IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)

X_temp = np.zeros((670, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_temp = np.zeros((670, IMG_HEIGHT, IMG_WIDTH), dtype=np.bool)

In [ ]:
# read all image train and train label
for n in range(670):
    imTrain = imread(TEST_PATH +'/im/'+ n.__str__() + '.png')
    imTrainLabel = imread(TEST_PATH +'/label/'+ n.__str__() + '.png')
    #img1 = np.array(img1)
    X_temp[n] = imTrain
    Y_temp[n] = imTrainLabel
  

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=368)


In [ ]:
# Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)


In [ ]:
# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.2)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.2)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.2)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

# Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.2)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.2)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
results = model.fit(X_train, y_train, validation_split=0.2, batch_size=4, epochs=40)

In [ ]:
resutl_path = 'segmented_result3/' # result directory for segmentation path
evaluate_result = model.evaluate(X_test,y_test)


5/5 [==============================] - 4s 249ms/step - loss: 0.0709 - accuracy: 0.9741


In [ ]:
evaluate_result

In [ ]:

numpy_accuracy= np.array(results.history['accuracy'])
np.savetxt("train_accuracy.txt", numpy_accuracy, delimiter=",")

numpy_val_accuracy= np.array(results.history['val_accuracy'])
np.savetxt("val_accuracy.txt", numpy_val_accuracy, delimiter=",")


numpy_loss= np.array(results.history['loss'])
np.savetxt("train_loss.txt", numpy_loss, delimiter=",")

numpy_val_loss= np.array(results.history['val_loss'])
np.savetxt("val_loss.txt", numpy_val_loss, delimiter=",")

In [ ]:
preds_test = model.predict(X_test, verbose=1)

5/5 [==============================] - 1s 86ms/step


In [ ]:
import matplotlib.cm as cm

In [ ]:
for n in range(134):
    imsave(resutl_path +'/test_orj/'+ n.__str__() + '.png', X_test[n])
#    imsave(resutl_path +'/test_orj_label/' + n.__str__() + '.png', y_test[n])
    imsave(resutl_path +'/test_pred/'+ n.__str__() + '.png', preds_test[n])
   
    

In [ ]:
for n in range(134):
    plt.imsave(resutl_path +'/test_orj_label/' + n.__str__() + '.png',y_test[n],cmap = cm.binary_r)
    
   

In [ ]:
preds_test = (preds_test > 0.35).astype(np.bool)
preds_test = preds_test.reshape((134,256,256))

for n in range(134):
    plt.imsave(resutl_path +'/test_pred_withThreshold/' + n.__str__() + '.png',preds_test[n],cmap = cm.binary_r)

